# Wiki Toronto postal codes data#

** Import packages**

In [1]:
from bs4 import BeautifulSoup
import requests
import csv

** Import the Data **

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className = document.documentElement.className.replace( /(^|\s)client-nojs(\s|$)/, "$1client-js$2" );
  </script>
  <script>
   (window.RLQ=window.RLQ||[]).push(function(){mw.config.set({"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":862527922,"wgRevisionId":862527922,"wgArticleId":539066,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Communications in Ontario","Postal codes in Canada","Toronto","Ontario-related lists"],"wgBreakFrames":false,"wgPageContentLanguage":"en","wgPageContentModel":"wikitext","wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wg

In [3]:
title = soup.title.text
print(title)

List of postal codes of Canada: M - Wikipedia


** Convert the table to a pandas Dataframe**

In [4]:
import pandas as pd
import numpy as np

dataSheet= []

for rows in soup.find_all('tr'):
    cell_content = rows.find_all('td')
    
    
    if len(cell_content) == 3:
        Postcode = cell_content[0].text.strip()
        Borough = cell_content[1].text.strip()
        Neighbourhood = cell_content[2].text.strip()
        
        
        if Borough == 'Not assigned':
            next
        else:  
            data = [Postcode, Borough, Neighbourhood]
            dataSheet.append(data)
df1 = pd.DataFrame(dataSheet, columns=['Postcode','Borough', 'Neighbourhood'])
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


**Replacing in neighborhoods that  are "Not assigned" with Name of the Borough  **

In [5]:
df1["Neighbourhood"] = df1["Neighbourhood"].replace("Not assigned", df1["Borough"])

** Combine the multiple neighborhoods**

In [6]:
df2 = df1.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

**Dataframe shape**

In [136]:
df2.shape

(103, 3)

**Checking if there are any "Not assigned" in "neighbourhood"**

In [137]:
df2.apply(lambda row: row.astype(str).str.contains('Not assigned').any(), axis=1)

0      False
1      False
2      False
3      False
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
30     False
31     False
32     False
33     False
34     False
35     False
36     False
37     False
38     False
39     False
40     False
41     False
42     False
43     False
44     False
45     False
46     False
47     False
48     False
49     False
50     False
51     False
52     False
53     False
54     False
55     False
56     False
57     False
58     False
59     False
60     False
61     False
62     False
63     False
64     False
65     False
66     False
67     False
68     False
69     False
70     False
71     False
72     False
73     False
74     False
75     False
76     False

** getting the corrdiniates of postal codes **

In [15]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]


ModuleNotFoundError: No module named 'geocoder'

In [16]:

coord = pd.read_csv('http://cocl.us/Geospatial_data')
# coord.columns = ['Postcode', 'Latitude', 'Longitude']
# columns = coord.columns
# for col in columns:
#     print(col)
coord[['Latitude', 'Longitude']].head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [19]:
df4 = df2.merge(coord.rename(columns={'Postal Code':'Postcode'}),how='left', on = ['Postcode'])
df4

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [75]:
df4.shape

(103, 5)

In [83]:
df5 = df2.merge(coord.rename(columns={'Postal Code':'Postcode'}),how='outer').reset_index()
df5.head()

,index,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [55]:
df5.shape

(103, 5)

In [113]:
north_york_df = df4[df4['Borough'].str.contains('North York')].reset_index(drop = True)

map_northyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, Neighbourhood in zip(north_york_df['Latitude'], north_york_df['Longitude'], north_york_df['Borough'], north_york_df['Neighbourhood']):
    label = '{}, {}'.format(Neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork)
map_northyork

In [116]:
north_york_df.loc[0, 'Neighbourhood']

'Hillcrest Village'